# PCA in Theano

I'm following along with this tutorial: https://lazyprogrammer.me/principal-components-analysis-in-theano/

In addition to helping me do the PSD analysis on the GPU, this will also give a gentle-ish intro to Theano...

## Variables and functions in Theano:

There's some weird allusions in the blog post to Theano all being based on Graph Theory, and how variables are nodes and equations are edges, or some such. Honestly I didn't follow, and I probably won't have to get into it in any case. In any case, conceptually it is just like thinking of Theano creating containers for variables and then defining function containers that operate on the variable containers to make new containers. The syntax seems simple enough once you see it:

In [36]:
import numpy as np
import theano
import theano.tensor as T
import pylearn2.models.pca
import sklearn.decomposition.pca
import torch

X = T.matrix('X')
Q = T.matrix('Q')
Z = T.dot(X, Q)

transform = theano.function(inputs=[X,Q], outputs=Z)

X_val = np.random.randn(100,10).astype(np.float32)
Q_val = np.random.randn(10,10).astype(np.float32)

Z_val = transform(X_val, Q_val)

PCA theory is not explained well in this blog post.
I'm going to black box it and if I feel the need, come back to it later...

In [5]:
Xv_val = np.dot(X_val, init_v)
np.dot(Xv_val.T, Xv_val)
np.sum(evals[j]*np.dot(evecs[j], init_v)*np.dot(evecs[j], init_v) for j in range(len(init_v)))

array([ 9.38270568])

In [6]:
cost.flatten()

Reshape{1}.0

In [4]:
init_v = np.random.randn(10,1)
v = theano.shared(init_v, name="v")
Xv = T.dot(X, v)
evals = np.random.randn(10)
evecs = np.random.randn(10,10)
cost = T.dot(Xv.T, Xv) - np.sum(evals[j]*T.dot(evecs[j], v)*T.dot(evecs[j], v) for j in range(len(init_v)))

gv = T.grad(cost.flatten(), v)

TypeError: cost must be a scalar.

OK, that doesn't seem to be working and his blog isn't detailed enough for me to figure it out. Time to move on to the official [Theano tutorial](http://deeplearning.net/software/theano/tutorial/index.html#tutorial)!

In [7]:
#Adding two scalars:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = theano.function([x,y], z)
print(f(2,3))

5.0


In [8]:
#Adding two matrices:
X = T.dmatrix('X')
Y = T.dmatrix('Y')
Z = X + Y
F = theano.function([X,Y], Z)
X_val = np.array([[0,1,2],[3,4,5]])
Y_val = np.array([[0,1,2],[3,4,5]])
print(F(X_val, Y_val))

[[  0.   2.   4.]
 [  6.   8.  10.]]


In [9]:
S = X.sum(axis=1)
fs = theano.function([X], S)
print(fs(X_val))

[  3.  12.]


In [10]:
BG = X[:, 0:2].sum(axis=1)/2
fbg = theano.function([X], BG)
print(fbg(X_val))

[ 0.5  3.5]


In [25]:
data = np.random.randn(10000, 1024).astype(np.float32)
#Try in numpy, as fast as possible...
%timeit data/(data[:, 40:].sum(axis=1) - data[:, 10:30].sum(axis=1)*((1024-40)/20))[:,None]

100 loops, best of 3: 11.4 ms per loop


In [26]:
X_norm = X/(X[:, 40:].sum(axis=1) - X[:, 10:30].sum(axis=1)*((1024-40)/20))[:, None]
fnorm = theano.function([X], X_norm)
%timeit fnorm(data)

10 loops, best of 3: 22.3 ms per loop


In [27]:
ret = fnorm(data)
type(ret)

numpy.ndarray

In [34]:
#Try sklearn pca
skpca = sklearn.decomposition.pca.PCA(n_components=10)
%timeit skpca.fit(ret)

1 loop, best of 3: 1.23 s per loop


In [35]:
#Try pylearn2 pca
plpca = pylearn2.models.pca.SVDPCA(num_components=10)
%timeit plpca.train(ret)

1 loop, best of 3: 1.55 s per loop


NOTE: Theano uses the CPU for the SVD!

skcuda can access the cusolver library, which does SVD on the GPU. I haven't been able to get the GPU to speed up the execution (to faster than numpy) using this approach, and have run into memory errors. In principle, it should be able to run faster, but I can't figure out why (I need to spend more time learning the basics)... For now I'll abandon this and use sklearn (which uses numpy).

Aside: the main processing part of doing PCA is SVD. With SVD it takes some matrix $A$ and return three matrices: [eigenvectors of $AA^{T}$ as columns] [sqrt of eigenvalues on diagonal (descending)] [eigenvectors of $A^{T}A$ as columns]. Subtracting the mean values of the waveforms means that $A^{T}A$ is just the covariance matrix, and so SVD gives us our PCA eigenvectors and eigenvalues in one step!